In [1]:
from dotenv import load_dotenv
import os
import io
import requests
from urllib.request import urlopen
import json
from geojson import Feature, FeatureCollection, Point
import numpy as np
import pandas as pd
import matplotlib as plt
import plotly.express as px
import plotly.graph_objects as go


In [2]:
# charger les tokens
load_dotenv('.env')
TOKEN = os.getenv("TOKEN")
mapbox = os.getenv("MAPBOX")

headers = {'x-api-key': TOKEN, 'Content-Type':'application/json', 'private': 'true'}

In [3]:
# definir l'adresse de l'API
api2 = 'https://data.education.gouv.fr/api/v2.1'
api1 = 'https://www.data.gouv.fr/api/1/'

In [4]:
# Open session
session = requests.Session()

# GET GEOJSON POUR REGIONS ACADEMIQUES

In [8]:
# Premier set de donnees pour obtenir le geoJSON des delimitations academiques de la France
r = session.get(f'{api2}/catalog/datasets/fr-en-contour-academies-2020/records').json()
# Obtenir l'adresse URL donnant acces au data set
href_academies = r['links'][3]['href']
# # Ouverture du data set
# with urlopen(href_academies) as response:
#     regions = json.load(response)

{'total_count': 30,
 'links': [{'rel': 'self',
   'href': 'https://data.education.gouv.fr/api/v2/catalog/datasets/fr-en-contour-academies-2020/records?limit=10&offset=0&include_app_metas=False&include_links=False'},
  {'rel': 'first',
   'href': 'https://data.education.gouv.fr/api/v2/catalog/datasets/fr-en-contour-academies-2020/records?limit=10&offset=0&include_app_metas=False&include_links=False'},
  {'rel': 'last',
   'href': 'https://data.education.gouv.fr/api/v2/catalog/datasets/fr-en-contour-academies-2020/records?limit=10&offset=20&include_app_metas=False&include_links=False'},
  {'rel': 'next',
   'href': 'https://data.education.gouv.fr/api/v2/catalog/datasets/fr-en-contour-academies-2020/records?limit=10&offset=10&include_app_metas=False&include_links=False'}],
 'records': [{'links': [{'rel': 'self',
     'href': 'https://data.education.gouv.fr/api/v2/catalog/datasets/fr-en-contour-academies-2020/records/cb907951db7cc440a77bfa7287db2e49e813f0a6'},
    {'rel': 'datasets',
     

# 1- Utiliser l'API pour accéder aux Datasets

DONEES EXTRAITES DU SITE :
https://data.education.gouv.fr/explore/?sort=modified&refine.keyword=coll%C3%A8ges

### 2 - Admin Express - Avec secteurs scolaires

In [6]:
r = session.get(f'{api1}datasets/60e678fececb26f4352c4f6a', headers = headers).json()
hrefs_csv = r['resources'][0]['latest']
hrefs_csv
with urlopen(hrefs_csv) as response:
    colleges = pd.read_csv(response, delimiter=';')

### 3 - Import du Dataset "Annuaire" (format csv) à partir duquel nous allons construire notre Dataframe ne comprenant que les données que nous souhaitons analyser et cartographier

Récupération du dataset Annuaire sous forme CSV

In [5]:
# Acceder au deuxieme data set
annuaire = session.get(f'{api2}/catalog/datasets/fr-en-annuaire-education/exports').json()
# Obtention du lien pour acceder au DF
href_annuaire = annuaire['links'][1]['href']
# ouverture
with urlopen(href_annuaire) as response:
    annuaire = pd.read_csv(response, delimiter=';')

/var/folders/5n/5_6qtt550_3dxz3f8y6wwjww0000gn/T/ipykernel_38038/4165119278.py:7: DtypeWarning: Columns (8,10,19,20,39,41,60,63) have mixed types. Specify dtype option on import or set low_memory=False.
  annuaire = pd.read_csv(response, delimiter=';')


In [12]:
#Dataframe à partir des données CSV
annuaire_df = pd.DataFrame(annuaire)
annuaire_df


,identifiant_de_l_etablissement,nom_etablissement,type_etablissement,statut_public_prive,adresse_1,adresse_2,adresse_3,code_postal,code_commune,nom_commune,...,rpi_concentre,rpi_disperse,code_nature,libelle_nature,code_type_contrat_prive,pial,etablissement_mere,type_rattachement_etablissement_mere,code_zone_animation_pedagogique,libelle_zone_animation_pedagogique
0,0800863D,Ecole maternelle Les Capucines,Ecole,Public,Rue Eugène Després,NaN,80440 BOVES,80440,80131,Boves,...,0.0,NaN,101,ECOLE MATERNELLE,99,0800038G,NaN,NaN,20802,AMIENS
1,0800867H,Ecole maternelle Valentine Clément,Ecole,Public,Place de la République,NaN,80170 ROSIERES EN SANTERRE,80170,80680,Rosières-en-Santerre,...,0.0,NaN,101,ECOLE MATERNELLE,99,0800051W,NaN,NaN,20803,SANTERRE SOMME
2,0800870L,Ecole maternelle Claude Monet,Ecole,Public,4 rue Denis Longuet,NaN,80190 NESLE,80190,80585,Nesle,...,0.0,NaN,101,ECOLE MATERNELLE,99,0801369D,NaN,NaN,20803,SANTERRE SOMME
3,0800873P,Ecole primaire Saint-Leu,Ecole,Public,35 rue de la Dodane,NaN,80000 AMIENS,80000,80021,Amiens,...,0.0,NaN,151,ECOLE DE NIVEAU ELEMENTAIRE,99,0800020M,NaN,NaN,20802,AMIENS
4,0800876T,Ecole primaire Beauvillé,Ecole,Public,400 boulevard de Beauville,NaN,80000 AMIENS,80000,80021,Amiens,...,0.0,NaN,151,ECOLE DE NIVEAU ELEMENTAIRE,99,0801373H,NaN,NaN,20802,AMIENS
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67628,0080771V,Ecole primaire,Ecole,Public,2 rue Lavoisier,NaN,08200 GLAIRE,8200,8194,Glaire,...,0.0,NaN,151,ECOLE DE NIVEAU ELEMENTAIRE,99,0080910W,NaN,NaN,19020,CHARLEVILLE-SEDAN
67629,0080781F,Ecole primaire Blanpain,Ecole,Public,10 RUE BLANPAIN,NaN,08200 SEDAN,8200,8409,Sedan,...,0.0,NaN,151,ECOLE DE NIVEAU ELEMENTAIRE,99,0080910W,NaN,NaN,19020,CHARLEVILLE-SEDAN
67630,0080794V,Ecole primaire Louise Michel,Ecole,Public,2 rue Jean Jaurès,NaN,08330 VRIGNE AUX BOIS,8330,8491,Vrigne aux Bois,...,0.0,NaN,151,ECOLE DE NIVEAU ELEMENTAIRE,99,0080839U,NaN,NaN,19020,CHARLEVILLE-SEDAN
67631,0080884T,Ecole primaire privée Sainte Thérèse,Ecole,Privé,61 rue Jean Jaurès,NaN,08700 NOUZONVILLE,8700,8328,Nouzonville,...,0.0,NaN,151,ECOLE DE NIVEAU ELEMENTAIRE,20,0080036W,NaN,NaN,19020,CHARLEVILLE-SEDAN


In [13]:
# ne prendre que les établissements "collège"
annuaire_clg_df = annuaire_df[annuaire_df['type_etablissement'] =='Collège']


In [10]:
# visualisation des noms des colonnes du Dataframe afin de voir lesquelles selectionner pour notre analyse
columns = annuaire_df.columns
#columns


### 1.1 SELECTION DES COLONNES : Dataframe avec les données choisies pour l'analyse et la cartographie

In [14]:
# Nous avons ici constitué un nouveau Dataframe sur lequel réaliser nos analyses lequel ne comprend que certaines colonnes du Dataframe initial "ANNUAIRE"
dispositifs_collèges_df = annuaire_clg_df[['code_academie','identifiant_de_l_etablissement', 'nom_etablissement',
    'type_etablissement','nom_commune','libelle_departement', 'libelle_academie','statut_public_prive','etat','nombre_d_eleves','hebergement','ulis','segpa','appartenance_education_prioritaire','latitude', 'longitude']]
dispositifs_collèges_df

,code_academie,identifiant_de_l_etablissement,nom_etablissement,type_etablissement,nom_commune,libelle_departement,libelle_academie,statut_public_prive,etat,nombre_d_eleves,hebergement,ulis,segpa,appartenance_education_prioritaire,latitude,longitude
2003,17,0440190T,Collège Sainte-Philomène,Collège,Coueron,Loire-Atlantique,Nantes,Privé,OUVERT,674.0,0.0,0.0,0.0,NaN,47.212258,-1.724250
2004,17,0720014Z,Collège André Pioger,Collège,Conlie,Sarthe,Nantes,Public,OUVERT,380.0,0.0,1.0,1.0,NaN,48.131339,-0.015090
2006,17,0441545R,Collège la Noe Lambert,Collège,Nantes,Loire-Atlantique,Nantes,Public,OUVERT,597.0,0.0,1.0,1.0,NaN,47.232017,-1.527280
2007,17,0530064G,Collège Saint-Martin,Collège,Montsûrs,Mayenne,Nantes,Privé,OUVERT,136.0,0.0,0.0,0.0,NaN,48.134792,-0.550167
2008,17,0440293E,Collège Robert Schuman,Collège,Châteaubriant,Loire-Atlantique,Nantes,Public,OUVERT,466.0,0.0,1.0,0.0,NaN,47.726093,-1.370831
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57292,41,9840458A,Centre des jeunes adolescents VAIRAO,Collège,Taiarapu-o,Polynésie Française,Polynésie Française,Public,OUVERT,NaN,0.0,0.0,1.0,NaN,NaN,NaN
62159,17,0851145H,Collège Marais Poitevin,Collège,Benet,Vendée,Nantes,Public,OUVERT,279.0,0.0,0.0,0.0,NaN,46.371361,-0.585580
62161,17,0442029S,Collège Gérard Philipe,Collège,Carquefou,Loire-Atlantique,Nantes,Public,OUVERT,825.0,0.0,0.0,0.0,NaN,47.283349,-1.483941
62162,17,0491260D,Collège François Villon,Collège,Les Ponts-de-Cé,Maine-et-Loire,Nantes,Public,OUVERT,660.0,0.0,1.0,0.0,NaN,47.432436,-0.521650


### Renommer la colonne 'identifier etablisssment' par 'numero_d_etablissement' afin qu'elle porte le meme nom que le dataframe IPS_reussite

In [15]:
dispositifs_collèges_df = dispositifs_collèges_df.rename(columns = {'identifiant_de_l_etablissement':'numero_d_etablissement'})

### 1.2 NETTOYAGE : Etant donné le manque de beaucoup de données concernant certaines académies, nous avons choisis de les retirer

### Identification des NaN et Nettoyage du DF

In [16]:
# compter le nomber de NaN dans toutes les colonnes
nan_columns = dispositifs_collèges_df.isna().sum()
nan_columns

code_academie                            0
numero_d_etablissement                   0
nom_etablissement                        0
type_etablissement                       0
nom_commune                              0
libelle_departement                      0
libelle_academie                         0
statut_public_prive                      0
etat                                     0
nombre_d_eleves                        642
hebergement                              0
ulis                                     0
segpa                                    0
appartenance_education_prioritaire    6533
latitude                               123
longitude                              123
dtype: int64

In [11]:
# liste des académies a retirer de mon DF : tous les DOM TOM + la corse
to_drop = ['Polynésie Française','La Réunion','Martinique','Guadeloupe', 'La Réunion',
       'Mayotte', 'Polynésie Française', 'Guyane',
       'Nouvelle Calédonie', 'Corse', 'Wallis et Futuna',
       'Saint Pierre et Miquelon']

In [15]:
#Par précaution : je commence par extraire la partie du dataframe avec "uniquement les académie contenues dans to_drop" pour savoir combien cela represente en nombre de lignes et pouvoir vérifier apres les avoir enlevé que j'ai bien retiré le bon nombre de lignes
dispositifs_collèges_df[dispositifs_collèges_df['libelle_academie'].isin(to_drop)]


,code_academie,numero_d_etablissement,nom_etablissement,type_etablissement,nom_commune,libelle_departement,libelle_academie,statut_public_prive,etat,nombre_d_eleves,hebergement,ulis,segpa,appartenance_education_prioritaire,latitude,longitude
2376,31,9720943T,Annexe collège privé saint Joseph de cluny,Collège,Sainte-Luce,Martinique,Martinique,Privé,OUVERT,NaN,0.0,0.0,1.0,NaN,14.469709,-60.919038
2652,32,9711249F,Collège ISAAC NEWTON,Collège,Saint-Martin,Saint-Martin,Guadeloupe,Privé,OUVERT,NaN,0.0,0.0,1.0,NaN,18.075875,-63.076070
2654,32,9711284U,BE HAPPY,Collège,Baie-Mahault,Guadeloupe,Guadeloupe,Privé,OUVERT,NaN,0.0,0.0,1.0,NaN,16.238230,-61.575425
2656,32,9711316D,Collège My First Academy,Collège,Le Gosier,Guadeloupe,Guadeloupe,Privé,OUVERT,NaN,0.0,0.0,1.0,NaN,16.210226,-61.470567
2657,31,9720925Y,COLLEGE PRIVE HORS CONTRAT DE LA PATIENCE,Collège,Schoelcher,Martinique,Martinique,Privé,OUVERT,NaN,0.0,0.0,1.0,NaN,14.628240,-61.109787
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57285,28,9741723E,Collège Privé Montessori Saint Joseph,Collège,Saint-Joseph,La Réunion,La Réunion,Privé,OUVERT,NaN,0.0,0.0,1.0,NaN,-21.377651,55.610964
57289,41,9840420J,CED DE TAIOHAE,Collège,Nuku-hiva,Polynésie Française,Polynésie Française,Privé,OUVERT,NaN,0.0,0.0,1.0,NaN,NaN,NaN
57290,41,9840440F,Centre des jeunes adolescents NAMAHA 4,Collège,Bora-bora,Polynésie Française,Polynésie Française,Public,OUVERT,NaN,0.0,0.0,1.0,NaN,NaN,NaN
57291,41,9840447N,Centre des jeunes adolescents PAEA AOU'A,Collège,Paea,Polynésie Française,Polynésie Française,Public,OUVERT,NaN,0.0,0.0,1.0,NaN,NaN,NaN


In [16]:
# query retirer toutes les lignes avec les académies contenues dans to_drop
dispositifs_collèges_df = dispositifs_collèges_df.query('libelle_academie not in @to_drop')
dispositifs_collèges_df

,code_academie,numero_d_etablissement,nom_etablissement,type_etablissement,nom_commune,libelle_departement,libelle_academie,statut_public_prive,etat,nombre_d_eleves,hebergement,ulis,segpa,appartenance_education_prioritaire,latitude,longitude
2003,17,0440190T,Collège Sainte-Philomène,Collège,Coueron,Loire-Atlantique,Nantes,Privé,OUVERT,674.0,0.0,0.0,0.0,NaN,47.212258,-1.724250
2004,17,0720014Z,Collège André Pioger,Collège,Conlie,Sarthe,Nantes,Public,OUVERT,380.0,0.0,1.0,1.0,NaN,48.131339,-0.015090
2006,17,0441545R,Collège la Noe Lambert,Collège,Nantes,Loire-Atlantique,Nantes,Public,OUVERT,597.0,0.0,1.0,1.0,NaN,47.232017,-1.527280
2007,17,0530064G,Collège Saint-Martin,Collège,Montsûrs,Mayenne,Nantes,Privé,OUVERT,136.0,0.0,0.0,0.0,NaN,48.134792,-0.550167
2008,17,0440293E,Collège Robert Schuman,Collège,Châteaubriant,Loire-Atlantique,Nantes,Public,OUVERT,466.0,0.0,1.0,0.0,NaN,47.726093,-1.370831
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57278,25,0952303M,Ecole secondaire privée - Collège Internationa...,Collège,Cergy,Val-d'Oise,Versailles,Privé,OUVERT,NaN,0.0,0.0,1.0,NaN,49.042775,2.055239
62159,17,0851145H,Collège Marais Poitevin,Collège,Benet,Vendée,Nantes,Public,OUVERT,279.0,0.0,0.0,0.0,NaN,46.371361,-0.585580
62161,17,0442029S,Collège Gérard Philipe,Collège,Carquefou,Loire-Atlantique,Nantes,Public,OUVERT,825.0,0.0,0.0,0.0,NaN,47.283349,-1.483941
62162,17,0491260D,Collège François Villon,Collège,Les Ponts-de-Cé,Maine-et-Loire,Nantes,Public,OUVERT,660.0,0.0,1.0,0.0,NaN,47.432436,-0.521650


In [17]:
# Les lignes correspondant aux académies de la liste to_drop ont bien été supprimées
#ci-dessous il ne me reste que les valeurs NAN correspondants aux nombre d'éleves ;  à l'appartenance education prioritaire ou non ; et seulement 4 valeurs NAN pour latitude et longitude (correspondant aux 3 académies paris, besançon et orléan_tours)
dispositifs_collèges_df.isna().sum()

code_academie                            0
numero_d_etablissement                   0
nom_etablissement                        0
type_etablissement                       0
nom_commune                              0
libelle_departement                      0
libelle_academie                         0
statut_public_prive                      0
etat                                     0
nombre_d_eleves                        501
hebergement                              0
ulis                                     0
segpa                                    0
appartenance_education_prioritaire    6256
latitude                                 4
longitude                                4
dtype: int64

### Colonne 'appartenance_education_prioritaire'
#### Remplacement des valeurs :
##### 0 --> Nan qu'on interprète comme : absence de données
##### 1 --> REP réseau d’éducation prioritaire 
##### 2 --> REP+ réseau d’éducation prioritaire renforcé 

In [18]:
# Remplacer les nan par 0
dispositifs_collèges_df['appartenance_education_prioritaire'] = dispositifs_collèges_df['appartenance_education_prioritaire'].fillna(0)


# vérification que les nan ont bien été remplacés par 0
dispositifs_collèges_df.isna().sum()
dispositifs_collèges_df['appartenance_education_prioritaire']

2003     0
2004     0
2006     0
2007     0
2008     0
        ..
57278    0
62159    0
62161    0
62162    0
62163    0
Name: appartenance_education_prioritaire, Length: 7204, dtype: object

In [19]:
# Remplacer REP et REP+

# par précaution je calcule le nombre de valeurs rep et rep+ dans ma colonne
dispositifs_collèges_df['appartenance_education_prioritaire'].value_counts()

# remplacer rep par 1 et rep+ par 2
dispositifs_collèges_df.replace({'appartenance_education_prioritaire': {'REP':1,'REP+':2}}, inplace = True)

# vérifiction que le remplace des valeurs de la colonne 'appartenance_education_prioritaire' a été bien éffectué
dispositifs_collèges_df['appartenance_education_prioritaire'].value_counts()


0    6256
1     660
2     288
Name: appartenance_education_prioritaire, dtype: int64

## cas particulier à traiter : nan nombre d'él_èves
#### le traitement de ces données nan est particulier car nous ne pouvons pas les remplacer par 0. Car nous auront besoin de la donnée nombre d'élèves pour établir la densité des élèves par académie et l'adéquation de l'offre en termes d'établissements collèges avec le nombre d'éleves dans chaque académie, ou région ou commune.

### Nous avons choisis de remplacer les valeurs NaN par la moyenne par académie en faisant la discinction entre etablissements publics et privés

In [19]:
grouped_acad_nbre = dispositifs_collèges_df.groupby(['code_academie','statut_public_prive'])['nombre_d_eleves'].mean().reset_index()
grouped_acad_nbre["nombre_d_eleves"] = round(grouped_acad_nbre["nombre_d_eleves"])
grouped_acad_nbre

,code_academie,statut_public_prive,nombre_d_eleves
0,1,Privé,475.0
1,1,Public,480.0
2,2,Privé,415.0
3,2,Public,567.0
4,3,Privé,284.0
...,...,...,...
62,43,Public,1355.0
63,44,Privé,NaN
64,44,Public,NaN
65,70,Privé,387.0


In [20]:
#Renommer la colonne 'nombre_d_eleves' (comprenant en réalité le calcul moyen du nombre d'élèves) par académie par 'nb_moyen"
grouped_acad_nbre.rename(columns={"nombre_d_eleves":"nb_moyen"}, inplace = True)

In [22]:
grouped_acad_nbre

,code_academie,statut_public_prive,nb_moyen
0,1,Privé,475.0
1,1,Public,480.0
2,2,Privé,415.0
3,2,Public,567.0
4,3,Privé,284.0
5,3,Public,448.0
6,4,Privé,360.0
7,4,Public,506.0
8,6,Privé,267.0
9,6,Public,344.0


In [21]:
#Réalisation du merge entre grouped et mon dataframe dispositifs
dispositifs_collèges_df_merged = dispositifs_collèges_df.merge(grouped_acad_nbre, on=["code_academie","statut_public_prive"])
dispositifs_collèges_df_merged

,code_academie,numero_d_etablissement,nom_etablissement,type_etablissement,nom_commune,libelle_departement,libelle_academie,statut_public_prive,etat,nombre_d_eleves,hebergement,ulis,segpa,appartenance_education_prioritaire,latitude,longitude,nb_moyen
0,17,0440190T,Collège Sainte-Philomène,Collège,Coueron,Loire-Atlantique,Nantes,Privé,OUVERT,674.0,0.0,0.0,0.0,NaN,47.212258,-1.724250,514.0
1,17,0530064G,Collège Saint-Martin,Collège,Montsûrs,Mayenne,Nantes,Privé,OUVERT,136.0,0.0,0.0,0.0,NaN,48.134792,-0.550167,514.0
2,17,0490851J,Collège Dom Sortais,Collège,Beaupréau-en-Mauges,Maine-et-Loire,Nantes,Privé,OUVERT,783.0,1.0,1.0,0.0,NaN,47.197597,-0.996870,514.0
3,17,0442858T,Ecole Secondaire privée International School o...,Collège,Saint-Herblain,Loire-Atlantique,Nantes,Privé,OUVERT,NaN,0.0,0.0,1.0,NaN,47.228024,-1.650231,514.0
4,17,0442870F,Ecole Secondaire Privée Graine d'Explorateur,Collège,Les Sorinières,Loire-Atlantique,Nantes,Privé,OUVERT,NaN,0.0,0.0,1.0,NaN,47.121788,-1.525586,514.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7627,42,9870032W,Collège Finemui de Teesi,Collège,Wallis,Wallis et Futuna,Wallis et Futuna,Public,OUVERT,NaN,0.0,0.0,0.0,NaN,NaN,NaN,29.0
7628,42,9870003P,Collège Saalauniu-Fiua,Collège,Sigave,Wallis et Futuna,Wallis et Futuna,Public,OUVERT,NaN,0.0,0.0,0.0,NaN,NaN,NaN,29.0
7629,44,9750025D,Collège Emile Letournel,Collège,Saint-Pierre,St-Pierre-et-Miquelon,Saint Pierre et Miquelon,Public,OUVERT,NaN,0.0,0.0,0.0,NaN,46.783514,-56.177265,NaN
7630,44,9750026E,Collège d'Etat du socle,Collège,Miquelon-Langlade,St-Pierre-et-Miquelon,Saint Pierre et Miquelon,Public,OUVERT,NaN,0.0,0.0,0.0,NaN,47.100576,-56.378860,NaN


In [22]:
dispositifs_collèges_df_merged.isnull().sum()

code_academie                            0
numero_d_etablissement                   0
nom_etablissement                        0
type_etablissement                       0
nom_commune                              0
libelle_departement                      0
libelle_academie                         0
statut_public_prive                      0
etat                                     0
nombre_d_eleves                        642
hebergement                              0
ulis                                     0
segpa                                    0
appartenance_education_prioritaire    6533
latitude                               123
longitude                              123
nb_moyen                                63
dtype: int64

## dispositifs_collèges_df_clean

### Remplacement des valeurs NaN de la colonne "nombre_d_eleves" par la moyenne du nombre d'élèves calculée pour l'académie correspodante

In [23]:
# Dataframe Final nettoyé
dispositifs_collèges_df_clean = dispositifs_collèges_df_merged

In [24]:
dispositifs_collèges_df_clean.isnull().sum()

code_academie                            0
numero_d_etablissement                   0
nom_etablissement                        0
type_etablissement                       0
nom_commune                              0
libelle_departement                      0
libelle_academie                         0
statut_public_prive                      0
etat                                     0
nombre_d_eleves                        642
hebergement                              0
ulis                                     0
segpa                                    0
appartenance_education_prioritaire    6533
latitude                               123
longitude                              123
nb_moyen                                63
dtype: int64

## 1.3 Finalisation du nottoyage du Dataframe Clean en remplaçant toutes les valeurs Nan de la colonne nombre d'élèves par la moyenne 

In [25]:
#Remplacement des NaN par la moyenne correspondante
dispositifs_collèges_df_clean["nombre_d_eleves"] = dispositifs_collèges_df_clean["nombre_d_eleves"].fillna(dispositifs_collèges_df_merged["nb_moyen"])

In [26]:
#Vérifications que j'ai remplacé toutes (501) les valeurs NaN de la colonne 'nombre_d_eleves'
dispositifs_collèges_df_clean["nombre_d_eleves"].isna().sum()

dispositifs_collèges_df_clean.isnull().sum()

code_academie                            0
numero_d_etablissement                   0
nom_etablissement                        0
type_etablissement                       0
nom_commune                              0
libelle_departement                      0
libelle_academie                         0
statut_public_prive                      0
etat                                     0
nombre_d_eleves                         63
hebergement                              0
ulis                                     0
segpa                                    0
appartenance_education_prioritaire    6533
latitude                               123
longitude                              123
nb_moyen                                63
dtype: int64

### 3 REGROUPER LES DATAFRAME ANNUAIRE-COLLEGE-DISPOSITIFS et LE DATAFRAME IPS-TAUX DE REUSSITE

#### Mise en place d'un data frame me permettant d'affiher des données relatives au taux de réussite du brevet par académie

In [18]:
js_brevet = session.get(f"{api1}datasets/5c3ead5e06e3e74fe24cc62c", headers = headers).json()
url_brevet = js_brevet['resources'][0]['url']
dfbrevet= pd.read_csv(url_brevet, delimiter=';')
dfbrevet = pd.DataFrame(dfbrevet)

In [30]:
dfbrevet

,session,numero_d_etablissement,denomination_principale,patronyme,secteur_d_enseignement,commune_et_arrondissement,commune_et_arrondissement_lib_l,departement,departement_libelle,academie,...,region,region_libelle,nombre_d_inscrits,nombre_de_presents,nombre_total_d_admis,nombre_d_admis_sans_mention,nombre_d_admis_mention_ab,nombre_d_admis_mention_b,nombre_d_admis_mention_tb,taux_de_reussite
0,2014,0131711P,COLLEGE,ROCHER DU DRAGON,PUBLIC,13001,AIX-EN-PROVENCE,013,BOUCHES-DU-RHONE,2,...,18,PROVENCE-ALPES-COTE D'AZUR,163,159,143,52,43,29,19,"89,90%"
1,2014,0131756N,COLLEGE,DARIUS MILHAUD,PUBLIC,13212,MARSEILLE 12E ARRONDISSEMENT,013,BOUCHES-DU-RHONE,2,...,18,PROVENCE-ALPES-COTE D'AZUR,153,143,109,49,25,21,14,"76,20%"
2,2014,0131923V,COLLEGE,MARSEILLEVEYRE,PUBLIC,13208,MARSEILLE 8E ARRONDISSEMENT,013,BOUCHES-DU-RHONE,2,...,18,PROVENCE-ALPES-COTE D'AZUR,262,256,236,63,81,59,33,"92,10%"
3,2014,0131943S,COLLEGE,PIERRE PUGET,PUBLIC,13206,MARSEILLE 6E ARRONDISSEMENT,013,BOUCHES-DU-RHONE,2,...,18,PROVENCE-ALPES-COTE D'AZUR,157,153,114,46,31,24,13,"74,50%"
4,2014,0131968U,COLLEGE,CAILLOLS (QUARTIER DES),PUBLIC,13212,MARSEILLE 12E ARRONDISSEMENT,013,BOUCHES-DU-RHONE,2,...,18,PROVENCE-ALPES-COTE D'AZUR,138,138,106,45,31,20,10,"76,80%"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
139575,2021,GROUPE12,AUTRE,-,-,-,-,-,-,12,...,6,GRAND EST,28,28,27,13,11,3,0,"96,4%"
139576,2021,GROUPE14,AUTRE,-,-,-,-,-,-,14,...,3,BRETAGNE,115,114,110,10,23,38,39,"96,5%"
139577,2021,GROUPE15,AUTRE,-,-,-,-,-,-,15,...,6,GRAND EST,3,3,3,0,3,0,0,"100,0%"
139578,2021,GROUPE25,AUTRE,-,-,-,-,-,-,25,...,10,ILE-DE-FRANCE,122,113,102,32,25,28,17,"90,3%"


In [27]:
# Sélectionner la dernière session disponible
df_brevet_2021 = dfbrevet[dfbrevet['session']==2021]

In [32]:
len(df_brevet_2021.columns)

21

## Mise en place d'un data frame me permettant d'afficher des données relatives à l'indice de position sociale par académie

L’indice de position sociale (IPS) des collèges, moyenne de l’IPS de leurs élèves, est un outil statistique élaboré par l’éducation nationale en 2016. Il permet de définir le profil social des établissements. À l’échelle de l’élève, l’IPS détermine sa position sociale en tenant compte de ses chances de réussite scolaire, à partir de la catégorie socioprofessionnelle de ses deux parents, leur profession, mais aussi les conditions matérielles, le capital culturel ou encore les pratiques culturelles. La finalité recherchée : savoir si ce contexte (plus ou moins favorable) influe sur la scolarité.
L’indice généré oscille entre 38 (un parent au chômage n’ayant jamais travaillé) et 179 (un parent instituteur ou ingénieur). Plus cet indice est haut, plus l’élève évolue dans un contexte familial favorable à sa réussite scolaire.

In [29]:
js_ips = session.get(f"{api1}datasets/63478da6e4e27dba6feacfa4", headers = headers).json()
url_ips = js_ips['resources'][0]['url']
dfips = pd.read_csv(url_ips, delimiter=';')
dfips = pd.DataFrame(dfips)

In [34]:
dfips

,rentree_scolaire,academie,code_du_departement,departement,uai,nom_de_l_etablissment,code_insee_de_la_commune,nom_de_la_commune,secteur,ips
0,2021-2022,LYON,001,AIN,0010039M,COLLEGE DE L ALBARINE,01384,SAINT RAMBERT EN BUGEY,public,98.5
1,2021-2022,LYON,001,AIN,0010084L,COLLEGE PRIVE SAINT CHARLES,01093,CHATILLON SUR CHALARONNE,privé sous contrat,114.6
2,2021-2022,LYON,001,AIN,0010794H,COLLEGE DU VALROMEY,01022,ARTEMARE,public,95.6
3,2021-2022,LYON,001,AIN,0010796K,COLLEGE EMILE CIZAIN,01262,MONTLUEL,public,115.2
4,2021-2022,LYON,001,AIN,0010820L,COLLEGE GEORGE SAND,01306,PONT DE VEYLE,public,104.2
...,...,...,...,...,...,...,...,...,...,...
6962,2021-2022,LA REUNION,974,LA REUNION,9741649Z,COLLEGE PRIVE CATHOLIQUE LEVAVASSEUR,97418,SAINTE MARIE,privé sous contrat,133.2
6963,2021-2022,MAYOTTE,976,MAYOTTE,9760042S,COLLEGE MARCEL HENRY,97606,CHIRONGUI,public,72.2
6964,2021-2022,MAYOTTE,976,MAYOTTE,9760180S,COLLEGE MARIAMA SALIM DE SADA,97616,SADA,public,82.1
6965,2021-2022,MAYOTTE,976,MAYOTTE,9760219J,COLLEGE DE M GOMBANI,97611,MAMOUDZOU,public,74.1


In [30]:

## je renomme la colonne 'uai' par 'numero_d_etablissment' pour pouvoir ensuite faire un merge avec mes deux dataframe
dfips.rename(columns = {'uai':'numero_d_etablissement'},inplace = True)
dfips

,rentree_scolaire,academie,code_du_departement,departement,numero_d_etablissement,nom_de_l_etablissment,code_insee_de_la_commune,nom_de_la_commune,secteur,ips
0,2021-2022,LYON,001,AIN,0010039M,COLLEGE DE L ALBARINE,01384,SAINT RAMBERT EN BUGEY,public,98.5
1,2021-2022,LYON,001,AIN,0010084L,COLLEGE PRIVE SAINT CHARLES,01093,CHATILLON SUR CHALARONNE,privé sous contrat,114.6
2,2021-2022,LYON,001,AIN,0010794H,COLLEGE DU VALROMEY,01022,ARTEMARE,public,95.6
3,2021-2022,LYON,001,AIN,0010796K,COLLEGE EMILE CIZAIN,01262,MONTLUEL,public,115.2
4,2021-2022,LYON,001,AIN,0010820L,COLLEGE GEORGE SAND,01306,PONT DE VEYLE,public,104.2
...,...,...,...,...,...,...,...,...,...,...
6962,2021-2022,LA REUNION,974,LA REUNION,9741649Z,COLLEGE PRIVE CATHOLIQUE LEVAVASSEUR,97418,SAINTE MARIE,privé sous contrat,133.2
6963,2021-2022,MAYOTTE,976,MAYOTTE,9760042S,COLLEGE MARCEL HENRY,97606,CHIRONGUI,public,72.2
6964,2021-2022,MAYOTTE,976,MAYOTTE,9760180S,COLLEGE MARIAMA SALIM DE SADA,97616,SADA,public,82.1
6965,2021-2022,MAYOTTE,976,MAYOTTE,9760219J,COLLEGE DE M GOMBANI,97611,MAMOUDZOU,public,74.1


In [36]:
len(dfips.columns)

10

### J'ai maintenant mes deux datasets: 
### - le premier relatif au taux de réussite au brevet par académie
### - le second relatif à l'ips des collèges par académie

### pour travailler plus en profondeur mes dataframes je vais lier avec merge() les datasets du brevet et de l'ips


In [55]:
dfmerged_IPS_réussite = dfips.merge(df_brevet_2021, left_on='numero_d_etablissement', right_on='numero_d_etablissement', validate='one_to_many')
dfmerged_IPS_réussite

,rentree_scolaire,academie_x,code_du_departement,departement_x,numero_d_etablissement,nom_de_l_etablissment,code_insee_de_la_commune,nom_de_la_commune,secteur,ips,...,region,region_libelle,nombre_d_inscrits,nombre_de_presents,nombre_total_d_admis,nombre_d_admis_sans_mention,nombre_d_admis_mention_ab,nombre_d_admis_mention_b,nombre_d_admis_mention_tb,taux_de_reussite
0,2021-2022,LYON,001,AIN,0010039M,COLLEGE DE L ALBARINE,01384,SAINT RAMBERT EN BUGEY,public,98.5,...,1,AUVERGNE-RHONE-ALPES,56,56,51,11,9,14,17,"91,1%"
1,2021-2022,LYON,001,AIN,0010084L,COLLEGE PRIVE SAINT CHARLES,01093,CHATILLON SUR CHALARONNE,privé sous contrat,114.6,...,1,AUVERGNE-RHONE-ALPES,56,56,52,8,19,13,12,"92,9%"
2,2021-2022,LYON,001,AIN,0010794H,COLLEGE DU VALROMEY,01022,ARTEMARE,public,95.6,...,1,AUVERGNE-RHONE-ALPES,84,83,74,16,17,18,23,"89,2%"
3,2021-2022,LYON,001,AIN,0010796K,COLLEGE EMILE CIZAIN,01262,MONTLUEL,public,115.2,...,1,AUVERGNE-RHONE-ALPES,173,172,152,15,32,32,73,"88,4%"
4,2021-2022,LYON,001,AIN,0010820L,COLLEGE GEORGE SAND,01306,PONT DE VEYLE,public,104.2,...,1,AUVERGNE-RHONE-ALPES,129,129,112,30,23,32,27,"86,8%"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6922,2021-2022,LA REUNION,974,LA REUNION,9741313J,COLLEGE JEAN LE TOULLEC,97407,LE PORT,public,64.5,...,11,LA REUNION,115,114,85,31,29,14,11,"74,6%"
6923,2021-2022,MAYOTTE,976,MAYOTTE,9760042S,COLLEGE MARCEL HENRY,97606,CHIRONGUI,public,72.2,...,13,MAYOTTE,234,232,179,84,63,24,8,"77,2%"
6924,2021-2022,MAYOTTE,976,MAYOTTE,9760180S,COLLEGE MARIAMA SALIM DE SADA,97616,SADA,public,82.1,...,13,MAYOTTE,236,234,179,88,49,31,11,"76,5%"
6925,2021-2022,MAYOTTE,976,MAYOTTE,9760219J,COLLEGE DE M GOMBANI,97611,MAMOUDZOU,public,74.1,...,13,MAYOTTE,420,413,324,157,78,57,32,"78,5%"


In [38]:
dfmerged_IPS_réussite.columns

#len(dfmerged_IPS_réussite.columns) = 30

Index(['rentree_scolaire', 'academie_x', 'code_du_departement',
       'departement_x', 'numero_d_etablissement', 'nom_de_l_etablissment',
       'code_insee_de_la_commune', 'nom_de_la_commune', 'secteur', 'ips',
       'session', 'denomination_principale', 'patronyme',
       'secteur_d_enseignement', 'commune_et_arrondissement',
       'commune_et_arrondissement_lib_l', 'departement_y',
       'departement_libelle', 'academie_y', 'academie_libelle', 'region',
       'region_libelle', 'nombre_d_inscrits', 'nombre_de_presents',
       'nombre_total_d_admis', 'nombre_d_admis_sans_mention',
       'nombre_d_admis_mention_ab', 'nombre_d_admis_mention_b',
       'nombre_d_admis_mention_tb', 'taux_de_reussite'],
      dtype='object')

## dfmerged_IPS_réussite_select 

### Retirer du DF IPS_reussites les colonnes dont nous n'avons pas besoin

In [56]:
dfmerged_IPS_réussite_select = dfmerged_IPS_réussite[['numero_d_etablissement','academie_y','academie_libelle','nombre_d_inscrits', 'nombre_de_presents','ips','secteur_d_enseignement','nombre_total_d_admis','nombre_d_admis_sans_mention','nombre_d_admis_mention_ab', 'nombre_d_admis_mention_b','nombre_d_admis_mention_tb','taux_de_reussite']]
#or df.loc[:,['A','B']]
dfmerged_IPS_réussite_select

,numero_d_etablissement,academie_y,academie_libelle,nombre_d_inscrits,nombre_de_presents,ips,secteur_d_enseignement,nombre_total_d_admis,nombre_d_admis_sans_mention,nombre_d_admis_mention_ab,nombre_d_admis_mention_b,nombre_d_admis_mention_tb,taux_de_reussite
0,0010039M,10,LYON,56,56,98.5,PUBLIC,51,11,9,14,17,"91,1%"
1,0010084L,10,LYON,56,56,114.6,PRIVE,52,8,19,13,12,"92,9%"
2,0010794H,10,LYON,84,83,95.6,PUBLIC,74,16,17,18,23,"89,2%"
3,0010796K,10,LYON,173,172,115.2,PUBLIC,152,15,32,32,73,"88,4%"
4,0010820L,10,LYON,129,129,104.2,PUBLIC,112,30,23,32,27,"86,8%"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6922,9741313J,28,LA REUNION,115,114,64.5,PUBLIC,85,31,29,14,11,"74,6%"
6923,9760042S,43,MAYOTTE,234,232,72.2,PUBLIC,179,84,63,24,8,"77,2%"
6924,9760180S,43,MAYOTTE,236,234,82.1,PUBLIC,179,88,49,31,11,"76,5%"
6925,9760219J,43,MAYOTTE,420,413,74.1,PUBLIC,324,157,78,57,32,"78,5%"


In [57]:
no_na = dfmerged_IPS_réussite_select.dropna()

In [58]:
no_na.columns

Index(['numero_d_etablissement', 'academie_y', 'academie_libelle',
       'nombre_d_inscrits', 'nombre_de_presents', 'ips',
       'secteur_d_enseignement', 'nombre_total_d_admis',
       'nombre_d_admis_sans_mention', 'nombre_d_admis_mention_ab',
       'nombre_d_admis_mention_b', 'nombre_d_admis_mention_tb',
       'taux_de_reussite'],
      dtype='object')

In [62]:
fig = px.scatter(no_na, x="ips", y="taux_de_reussite",
	color="academie_libelle",
    log_x=True, size_max=60)
fig.show()

In [33]:
dfmerged_IPS_réussite_select[['academie_y', 'academie_libelle']]

,academie_y,academie_libelle
0,10,LYON
1,10,LYON
2,10,LYON
3,10,LYON
4,10,LYON
...,...,...
6922,28,LA REUNION
6923,43,MAYOTTE
6924,43,MAYOTTE
6925,43,MAYOTTE


In [34]:
dfmerged_IPS_réussite_select['secteur_d_enseignement'].value_counts()

PUBLIC    5283
PRIVE     1644
Name: secteur_d_enseignement, dtype: int64

In [42]:
#dispositifs_collèges_df_clean["libelle_academie"].unique()
#len(dispositifs_collèges_df_clean["libelle_academie"].unique())

### Retirer du Dataframe IPS_reussite les académies dont nous n'avons pas les données 

In [35]:
# Retirer les académies avec valeurs manquantes du dataframe IPS_réussite
# liste des académies a retirer du DF : tous les DOM TOM + la corse
to_drop_2 = ['CORSE','GUADELOUPE', 'MARTINIQUE', 'GUYANE', 'LA REUNION', 'MAYOTTE']
# query retirer toutes les lignes avec les académies contenues dans to_drop
dfmerged_IPS_réussite_select_clean = dfmerged_IPS_réussite_select.query('academie_libelle not in @to_drop_2')
dfmerged_IPS_réussite_select_clean


,numero_d_etablissement,academie_y,academie_libelle,nombre_d_inscrits,nombre_de_presents,ips,secteur_d_enseignement,nombre_total_d_admis,nombre_d_admis_sans_mention,nombre_d_admis_mention_ab,nombre_d_admis_mention_b,nombre_d_admis_mention_tb,taux_de_reussite
0,0010039M,10,LYON,56,56,98.5,PUBLIC,51,11,9,14,17,"91,1%"
1,0010084L,10,LYON,56,56,114.6,PRIVE,52,8,19,13,12,"92,9%"
2,0010794H,10,LYON,84,83,95.6,PUBLIC,74,16,17,18,23,"89,2%"
3,0010796K,10,LYON,173,172,115.2,PUBLIC,152,15,32,32,73,"88,4%"
4,0010820L,10,LYON,129,129,104.2,PUBLIC,112,30,23,32,27,"86,8%"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6876,0951823R,25,VERSAILLES,104,101,125.0,PUBLIC,97,24,22,22,29,"96,0%"
6877,0951909J,25,VERSAILLES,152,151,101.2,PUBLIC,115,37,32,25,21,"76,2%"
6878,0951991Y,25,VERSAILLES,124,124,120.0,PUBLIC,112,27,35,28,22,"90,3%"
6879,0952080V,25,VERSAILLES,133,133,126.2,PUBLIC,126,33,23,22,48,"94,7%"


#### Renommer la colonne 'academie_y' par 'code_academie'

In [36]:
# Renommer la colonne 'academie_y' du Dataframe IPS_réussite par le même nom que le deuxieme Dataframe Dispositifs 'code_academie' afin de pouvoir réaliser le Merge sur cette colonne (seule colonne commune entre les 2)
dfmerged_IPS_réussite_select= dfmerged_IPS_réussite_select.rename(columns = {'academie_y':'code_academie'})

In [45]:
dfmerged_IPS_réussite_select.columns

Index(['numero_d_etablissement', 'code_academie', 'academie_libelle',
       'nombre_d_inscrits', 'nombre_de_presents', 'ips',
       'secteur_d_enseignement', 'nombre_total_d_admis',
       'nombre_d_admis_sans_mention', 'nombre_d_admis_mention_ab',
       'nombre_d_admis_mention_b', 'nombre_d_admis_mention_tb',
       'taux_de_reussite'],
      dtype='object')

### Nous avons maintenant nos deux Principaux DataFrames "Dispositifs_collège" et "Ips_Taux de réussite" que nous allons réunir dans un même grand Dataframe en réalisant un Merge sur la colonne : 'code academie' qui est nommées 'academie_y' dans DF IPS_reussite

In [46]:
dispositifs_collèges_df_clean

,code_academie,numero_d_etablissement,nom_etablissement,type_etablissement,nom_commune,libelle_departement,libelle_academie,statut_public_prive,etat,nombre_d_eleves,hebergement,ulis,segpa,appartenance_education_prioritaire,latitude,longitude,nb_moyen
0,17,0440190T,Collège Sainte-Philomène,Collège,Coueron,Loire-Atlantique,Nantes,Privé,OUVERT,674.0,0.0,0.0,0.0,0,47.212258,-1.724250,514.0
1,17,0530064G,Collège Saint-Martin,Collège,Montsûrs,Mayenne,Nantes,Privé,OUVERT,136.0,0.0,0.0,0.0,0,48.134792,-0.550167,514.0
2,17,0490851J,Collège Dom Sortais,Collège,Beaupréau-en-Mauges,Maine-et-Loire,Nantes,Privé,OUVERT,783.0,1.0,1.0,0.0,0,47.197597,-0.996870,514.0
3,17,0442858T,Ecole Secondaire privée International School o...,Collège,Saint-Herblain,Loire-Atlantique,Nantes,Privé,OUVERT,514.0,0.0,0.0,1.0,0,47.228024,-1.650231,514.0
4,17,0442870F,Ecole Secondaire Privée Graine d'Explorateur,Collège,Les Sorinières,Loire-Atlantique,Nantes,Privé,OUVERT,514.0,0.0,0.0,1.0,0,47.121788,-1.525586,514.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7199,22,0870865T,Ensemble scolaire Charles de Foucauld - site J...,Collège,Limoges,Haute-Vienne,Limoges,Privé,OUVERT,591.0,1.0,1.0,0.0,0,45.823531,1.265530,320.0
7200,22,0870909R,Collège Beaupeyrat,Collège,Limoges,Haute-Vienne,Limoges,Privé,OUVERT,428.0,1.0,0.0,0.0,0,45.827649,1.256051,320.0
7201,22,0190780V,Ensemble scolaire Edmond Michelet-site Bossuet...,Collège,Brive-la-Gaillarde,Corrèze,Limoges,Privé,OUVERT,528.0,1.0,0.0,1.0,0,45.157932,1.522329,320.0
7202,22,0190059L,Ensemble scolaire Edmond Michelet - site Notre...,Collège,Brive-la-Gaillarde,Corrèze,Limoges,Privé,OUVERT,478.0,1.0,0.0,0.0,0,45.155486,1.532570,320.0


In [47]:
dispositifs_collèges_df_clean.columns

Index(['code_academie', 'numero_d_etablissement', 'nom_etablissement',
       'type_etablissement', 'nom_commune', 'libelle_departement',
       'libelle_academie', 'statut_public_prive', 'etat', 'nombre_d_eleves',
       'hebergement', 'ulis', 'segpa', 'appartenance_education_prioritaire',
       'latitude', 'longitude', 'nb_moyen'],
      dtype='object')

#### on retire encore des colonnes dont nous n'avons plus besoin du Dataframe dispositifs_collèges_df_clean

In [37]:
dispositifs_collèges_df_clean= dispositifs_collèges_df_clean[['code_academie', 'numero_d_etablissement', 'nom_etablissement',
'libelle_academie', 'type_etablissement', 'statut_public_prive','nombre_d_eleves', 'hebergement', 'ulis', 'segpa','appartenance_education_prioritaire','latitude','longitude']]

In [49]:
dispositifs_collèges_df_clean.columns

Index(['code_academie', 'numero_d_etablissement', 'nom_etablissement',
       'libelle_academie', 'type_etablissement', 'statut_public_prive',
       'nombre_d_eleves', 'hebergement', 'ulis', 'segpa',
       'appartenance_education_prioritaire', 'latitude', 'longitude'],
      dtype='object')

In [50]:
dispositifs_collèges_df_clean

,code_academie,numero_d_etablissement,nom_etablissement,libelle_academie,type_etablissement,statut_public_prive,nombre_d_eleves,hebergement,ulis,segpa,appartenance_education_prioritaire,latitude,longitude
0,17,0440190T,Collège Sainte-Philomène,Nantes,Collège,Privé,674.0,0.0,0.0,0.0,0,47.212258,-1.724250
1,17,0530064G,Collège Saint-Martin,Nantes,Collège,Privé,136.0,0.0,0.0,0.0,0,48.134792,-0.550167
2,17,0490851J,Collège Dom Sortais,Nantes,Collège,Privé,783.0,1.0,1.0,0.0,0,47.197597,-0.996870
3,17,0442858T,Ecole Secondaire privée International School o...,Nantes,Collège,Privé,514.0,0.0,0.0,1.0,0,47.228024,-1.650231
4,17,0442870F,Ecole Secondaire Privée Graine d'Explorateur,Nantes,Collège,Privé,514.0,0.0,0.0,1.0,0,47.121788,-1.525586
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7199,22,0870865T,Ensemble scolaire Charles de Foucauld - site J...,Limoges,Collège,Privé,591.0,1.0,1.0,0.0,0,45.823531,1.265530
7200,22,0870909R,Collège Beaupeyrat,Limoges,Collège,Privé,428.0,1.0,0.0,0.0,0,45.827649,1.256051
7201,22,0190780V,Ensemble scolaire Edmond Michelet-site Bossuet...,Limoges,Collège,Privé,528.0,1.0,0.0,1.0,0,45.157932,1.522329
7202,22,0190059L,Ensemble scolaire Edmond Michelet - site Notre...,Limoges,Collège,Privé,478.0,1.0,0.0,0.0,0,45.155486,1.532570


In [51]:
dispositifs_collèges_df_clean["statut_public_prive"].value_counts()

Public    5093
Privé     2111
Name: statut_public_prive, dtype: int64

### Réalisatin du MERGE avec annuaire

In [52]:
#df_merged_TEST_a = dispositifs_collèges_df_clean.merge(dfmerged_IPS_réussite_select, on=["numero_d_etablissement"])

In [53]:
#df_merged_TEST_a

In [38]:
df_final_dispositifs_IPS_reussite = dispositifs_collèges_df_clean.merge(dfmerged_IPS_réussite_select, on=["numero_d_etablissement"], how = 'left')


In [55]:
df_final_dispositifs_IPS_reussite

,code_academie_x,numero_d_etablissement,nom_etablissement,libelle_academie,type_etablissement,statut_public_prive,nombre_d_eleves,hebergement,ulis,segpa,...,nombre_d_inscrits,nombre_de_presents,ips,secteur_d_enseignement,nombre_total_d_admis,nombre_d_admis_sans_mention,nombre_d_admis_mention_ab,nombre_d_admis_mention_b,nombre_d_admis_mention_tb,taux_de_reussite
0,17,0440190T,Collège Sainte-Philomène,Nantes,Collège,Privé,674.0,0.0,0.0,0.0,...,143.0,143.0,122.2,PRIVE,139.0,24.0,32.0,56.0,27.0,"97,2%"
1,17,0530064G,Collège Saint-Martin,Nantes,Collège,Privé,136.0,0.0,0.0,0.0,...,39.0,39.0,103.6,PRIVE,36.0,6.0,6.0,14.0,10.0,"92,3%"
2,17,0490851J,Collège Dom Sortais,Nantes,Collège,Privé,783.0,1.0,1.0,0.0,...,193.0,192.0,103.9,PRIVE,188.0,37.0,57.0,59.0,35.0,"97,9%"
3,17,0442858T,Ecole Secondaire privée International School o...,Nantes,Collège,Privé,514.0,0.0,0.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,17,0442870F,Ecole Secondaire Privée Graine d'Explorateur,Nantes,Collège,Privé,514.0,0.0,0.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7199,22,0870865T,Ensemble scolaire Charles de Foucauld - site J...,Limoges,Collège,Privé,591.0,1.0,1.0,0.0,...,149.0,149.0,128.7,PRIVE,148.0,10.0,27.0,44.0,67.0,"99,3%"
7200,22,0870909R,Collège Beaupeyrat,Limoges,Collège,Privé,428.0,1.0,0.0,0.0,...,98.0,97.0,120.6,PRIVE,94.0,27.0,26.0,26.0,15.0,"96,9%"
7201,22,0190780V,Ensemble scolaire Edmond Michelet-site Bossuet...,Limoges,Collège,Privé,528.0,1.0,0.0,1.0,...,126.0,126.0,112.8,PRIVE,110.0,32.0,30.0,30.0,18.0,"87,3%"
7202,22,0190059L,Ensemble scolaire Edmond Michelet - site Notre...,Limoges,Collège,Privé,478.0,1.0,0.0,0.0,...,120.0,120.0,124.9,PRIVE,114.0,24.0,22.0,41.0,27.0,"95,0%"


In [56]:
#df_final_dispositifs_IPS_reussite["type_etablissement"].unique()

In [57]:
#df_merged_TEST_2 = dispositifs_collèges_df_clean.merge(dfmerged_IPS_réussite_select, on=["code_academie"], how = 'left')
#df_merged_TEST_2 = dispositifs_collèges_df_clean.merge(dfmerged_IPS_réussite_select, on=["code_academie"])
#df_merged_TEST_2

In [58]:
df_final_dispositifs_IPS_reussite.columns

Index(['code_academie_x', 'numero_d_etablissement', 'nom_etablissement',
       'libelle_academie', 'type_etablissement', 'statut_public_prive',
       'nombre_d_eleves', 'hebergement', 'ulis', 'segpa',
       'appartenance_education_prioritaire', 'latitude', 'longitude',
       'code_academie_y', 'academie_libelle', 'nombre_d_inscrits',
       'nombre_de_presents', 'ips', 'secteur_d_enseignement',
       'nombre_total_d_admis', 'nombre_d_admis_sans_mention',
       'nombre_d_admis_mention_ab', 'nombre_d_admis_mention_b',
       'nombre_d_admis_mention_tb', 'taux_de_reussite'],
      dtype='object')

In [39]:
df_final_dispositifs_IPS_reussite['academie_libelle']

0       NANTES
1       NANTES
2       NANTES
3          NaN
4          NaN
         ...  
7627       NaN
7628       NaN
7629       NaN
7630       NaN
7631       NaN
Name: academie_libelle, Length: 7632, dtype: object

## Ajout de la colonne Zones A, B, C



Article 3 - Les académies, à l'exception de celles visées à l'article 5, sont réparties en trois zones de vacances A, B et C.

La zone A comprend les académies de Besançon, Bordeaux, Clermont-Ferrand, Dijon, Grenoble, Limoges, Lyon et Poitiers.

La zone B comprend les académies d'Aix-Marseille, Amiens, Lille, Nancy-Metz, Nantes, Nice, Normandie, Orléans-Tours, Reims, Rennes et Strasbourg.

La zone C comprend les académies de Créteil, Montpellier, Paris, Toulouse et Versailles.

  

In [40]:
df_final_dispositifs_IPS_reussite['libelle_academie'].unique()

array(['Nantes', 'Lyon', 'Normandie', 'Lille', 'Reims', 'Poitiers',
       'Grenoble', 'Strasbourg', 'Toulouse', 'Rennes', 'Nice',
       'Clermont-Ferrand', 'Créteil', 'Martinique', 'Orléans-Tours',
       'Paris', 'Versailles', 'Amiens', 'Montpellier', 'Aix-Marseille',
       'Dijon', 'Bordeaux', 'Nancy-Metz', 'Guadeloupe', 'La Réunion',
       'Mayotte', 'Polynésie Française', 'Besançon', 'Limoges', 'Guyane',
       'Nouvelle Calédonie', 'Corse', 'Wallis et Futuna',
       'Saint Pierre et Miquelon'], dtype=object)

In [41]:
# Should we work with ZONES ?
a = ['Besançon','Bordeaux','Clermont-Ferrand','Dijon','Grenoble','Limoges','Lyon','Poitiers']

b = ['Aix-Marseille','Amiens','Lille','Nancy-Metz','Nantes','Nice','Normandie','Orléans-Tours','Reims','Rennes','Strasbourg']

c = ['Créteil','Montpellier','Paris','Toulouse','Versailles']
# Create conditions and values to add 'zone' column
conditions = [
    (df_final_dispositifs_IPS_reussite['libelle_academie'].isin(a)),
    (df_final_dispositifs_IPS_reussite['libelle_academie'].isin(b)),
    (df_final_dispositifs_IPS_reussite['libelle_academie'].isin(c))
]

values = ['A', 'B', 'C']
# create 'zone' column
df_final_dispositifs_IPS_reussite['zone'] = np.select(conditions, values)

In [62]:
df_final_dispositifs_IPS_reussite

,code_academie_x,numero_d_etablissement,nom_etablissement,libelle_academie,type_etablissement,statut_public_prive,nombre_d_eleves,hebergement,ulis,segpa,...,nombre_de_presents,ips,secteur_d_enseignement,nombre_total_d_admis,nombre_d_admis_sans_mention,nombre_d_admis_mention_ab,nombre_d_admis_mention_b,nombre_d_admis_mention_tb,taux_de_reussite,zone
0,17,0440190T,Collège Sainte-Philomène,Nantes,Collège,Privé,674.0,0.0,0.0,0.0,...,143.0,122.2,PRIVE,139.0,24.0,32.0,56.0,27.0,"97,2%",B
1,17,0530064G,Collège Saint-Martin,Nantes,Collège,Privé,136.0,0.0,0.0,0.0,...,39.0,103.6,PRIVE,36.0,6.0,6.0,14.0,10.0,"92,3%",B
2,17,0490851J,Collège Dom Sortais,Nantes,Collège,Privé,783.0,1.0,1.0,0.0,...,192.0,103.9,PRIVE,188.0,37.0,57.0,59.0,35.0,"97,9%",B
3,17,0442858T,Ecole Secondaire privée International School o...,Nantes,Collège,Privé,514.0,0.0,0.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,B
4,17,0442870F,Ecole Secondaire Privée Graine d'Explorateur,Nantes,Collège,Privé,514.0,0.0,0.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,B
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7199,22,0870865T,Ensemble scolaire Charles de Foucauld - site J...,Limoges,Collège,Privé,591.0,1.0,1.0,0.0,...,149.0,128.7,PRIVE,148.0,10.0,27.0,44.0,67.0,"99,3%",A
7200,22,0870909R,Collège Beaupeyrat,Limoges,Collège,Privé,428.0,1.0,0.0,0.0,...,97.0,120.6,PRIVE,94.0,27.0,26.0,26.0,15.0,"96,9%",A
7201,22,0190780V,Ensemble scolaire Edmond Michelet-site Bossuet...,Limoges,Collège,Privé,528.0,1.0,0.0,1.0,...,126.0,112.8,PRIVE,110.0,32.0,30.0,30.0,18.0,"87,3%",A
7202,22,0190059L,Ensemble scolaire Edmond Michelet - site Notre...,Limoges,Collège,Privé,478.0,1.0,0.0,0.0,...,120.0,124.9,PRIVE,114.0,24.0,22.0,41.0,27.0,"95,0%",A


In [63]:
df_final_dispositifs_IPS_reussite.columns

Index(['code_academie_x', 'numero_d_etablissement', 'nom_etablissement',
       'libelle_academie', 'type_etablissement', 'statut_public_prive',
       'nombre_d_eleves', 'hebergement', 'ulis', 'segpa',
       'appartenance_education_prioritaire', 'latitude', 'longitude',
       'code_academie_y', 'academie_libelle', 'nombre_d_inscrits',
       'nombre_de_presents', 'ips', 'secteur_d_enseignement',
       'nombre_total_d_admis', 'nombre_d_admis_sans_mention',
       'nombre_d_admis_mention_ab', 'nombre_d_admis_mention_b',
       'nombre_d_admis_mention_tb', 'taux_de_reussite', 'zone'],
      dtype='object')

### TRAVAIL SUR LE DATAFRAME FINAL COnTENANT TOUTES LES DONNEES REGROUPEES
Nous avons jusqu'ici récolté plusieurs types de données à partir de plusieurs sources (plusieurs datasets que nous avons transformé en dataframes)
le dataframe final contient entre autre les données suivantes:

numero_d_etablissement	
code et libelle_academie
nombre_d_eleves	
hebergement	
ulis	
segpa
appartenance_education_prioritaire
ips
taux_de_reussite	
zone
latitude
longitude

Ce sont ces données qui nous interessent le plus et sur lequelles nous allons nous baser pour réaliser la carte.


### Sous Dataframe avec les colonnes des valeurs à cartographier

In [42]:
df_data_carto = df_final_dispositifs_IPS_reussite[['numero_d_etablissement', 'nom_etablissement',
    'code_academie_y','libelle_academie','statut_public_prive',
    'nombre_d_eleves', 'hebergement', 'ulis', 'segpa',
    'appartenance_education_prioritaire', 'ips','taux_de_reussite',
    'latitude', 'longitude','zone']]

In [65]:
df_data_carto

,numero_d_etablissement,nom_etablissement,code_academie_y,libelle_academie,statut_public_prive,nombre_d_eleves,hebergement,ulis,segpa,appartenance_education_prioritaire,ips,taux_de_reussite,latitude,longitude,zone
0,0440190T,Collège Sainte-Philomène,17.0,Nantes,Privé,674.0,0.0,0.0,0.0,0,122.2,"97,2%",47.212258,-1.724250,B
1,0530064G,Collège Saint-Martin,17.0,Nantes,Privé,136.0,0.0,0.0,0.0,0,103.6,"92,3%",48.134792,-0.550167,B
2,0490851J,Collège Dom Sortais,17.0,Nantes,Privé,783.0,1.0,1.0,0.0,0,103.9,"97,9%",47.197597,-0.996870,B
3,0442858T,Ecole Secondaire privée International School o...,NaN,Nantes,Privé,514.0,0.0,0.0,1.0,0,NaN,NaN,47.228024,-1.650231,B
4,0442870F,Ecole Secondaire Privée Graine d'Explorateur,NaN,Nantes,Privé,514.0,0.0,0.0,1.0,0,NaN,NaN,47.121788,-1.525586,B
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7199,0870865T,Ensemble scolaire Charles de Foucauld - site J...,22.0,Limoges,Privé,591.0,1.0,1.0,0.0,0,128.7,"99,3%",45.823531,1.265530,A
7200,0870909R,Collège Beaupeyrat,22.0,Limoges,Privé,428.0,1.0,0.0,0.0,0,120.6,"96,9%",45.827649,1.256051,A
7201,0190780V,Ensemble scolaire Edmond Michelet-site Bossuet...,22.0,Limoges,Privé,528.0,1.0,0.0,1.0,0,112.8,"87,3%",45.157932,1.522329,A
7202,0190059L,Ensemble scolaire Edmond Michelet - site Notre...,22.0,Limoges,Privé,478.0,1.0,0.0,0.0,0,124.9,"95,0%",45.155486,1.532570,A


#### Renommer la colonne 'code_academie_y' par 'code_academie'

In [43]:
df_data_carto=df_data_carto.rename(columns = {'code_academie_y':'code_academie'})

#### Ne garder que les établissements publics

In [67]:
df_data_carto['statut_public_prive'].value_counts()

Public    5093
Privé     2111
Name: statut_public_prive, dtype: int64

In [44]:
df_data_carto = df_data_carto.query('statut_public_prive == "Public"')
df_data_carto

,numero_d_etablissement,nom_etablissement,code_academie,libelle_academie,statut_public_prive,nombre_d_eleves,hebergement,ulis,segpa,appartenance_education_prioritaire,ips,taux_de_reussite,latitude,longitude,zone
191,0720014Z,Collège André Pioger,17.0,Nantes,Public,380.0,0.0,1.0,1.0,NaN,100.3,"96,6%",48.131339,-0.015090,B
192,0441545R,Collège la Noe Lambert,17.0,Nantes,Public,597.0,0.0,1.0,1.0,NaN,101.7,"79,0%",47.232017,-1.527280,B
193,0440293E,Collège Robert Schuman,17.0,Nantes,Public,466.0,0.0,1.0,0.0,NaN,93.6,"78,1%",47.726093,-1.370831,B
194,0440025N,Collège Hector Berlioz,17.0,Nantes,Public,456.0,0.0,1.0,0.0,NaN,110.4,"86,8%",47.240490,-1.571640,B
195,0442417N,Collège Les Sables d'or,17.0,Nantes,Public,614.0,0.0,1.0,0.0,NaN,123.4,"97,2%",47.264482,-1.456400,B
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7626,9870016D,Collège Mataotama,NaN,Wallis et Futuna,Public,29.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,0
7627,9870032W,Collège Finemui de Teesi,NaN,Wallis et Futuna,Public,29.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,0
7628,9870003P,Collège Saalauniu-Fiua,NaN,Wallis et Futuna,Public,29.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,0
7629,9750025D,Collège Emile Letournel,NaN,Saint Pierre et Miquelon,Public,NaN,0.0,0.0,0.0,NaN,NaN,NaN,46.783514,-56.177265,0


In [51]:
no_na_carto = df_data_carto.dropna()

In [52]:
fig = px.scatter(no_na_carto, x="ips", y="taux_de_reussite",
	    size="nombre_d_eleves", color="libelle_academie",
        hover_name="libelle_academie", log_x=True, size_max=60)
fig.show()

In [45]:
df_data_carto.isna().sum()

numero_d_etablissement                   0
nom_etablissement                        0
code_academie                          144
libelle_academie                         0
statut_public_prive                      0
nombre_d_eleves                         38
hebergement                              0
ulis                                     0
segpa                                    0
appartenance_education_prioritaire    4337
ips                                    144
taux_de_reussite                       144
latitude                                90
longitude                               90
zone                                     0
dtype: int64

In [83]:
no_services = df_data_carto.query("appartenance_education_prioritaire == 0 & segpa == 0 & ulis == 0")['libelle_academie'].reset_index()

In [84]:
no_services

,index,libelle_academie
0,196,Nantes
1,199,Nantes
2,209,Nantes
3,210,Nantes
4,211,Nantes
...,...,...
1608,7182,Besançon
1609,7183,Besançon
1610,7184,Besançon
1611,7189,Besançon


In [87]:
no_services = no_services.groupby("libelle_academie").count().reset_index()

In [89]:
no_services

,libelle_academie,index
0,Aix-Marseille,64
1,Amiens,45
2,Besançon,35
3,Bordeaux,94
4,Clermont-Ferrand,61
5,Créteil,93
6,Dijon,73
7,Grenoble,88
8,Lille,84
9,Limoges,47


In [92]:
no_services['ratio'] = round((no_services['index']/1613)*100,2)

In [96]:
no_services.sort_values(by='ratio')

,libelle_academie,index,ratio
17,Paris,18,1.12
21,Strasbourg,32,1.98
10,Lyon,32,1.98
2,Besançon,35,2.17
19,Reims,44,2.73
1,Amiens,45,2.79
16,Orléans-Tours,47,2.91
9,Limoges,47,2.91
4,Clermont-Ferrand,61,3.78
0,Aix-Marseille,64,3.97


### CARTOGRAPHIE DES DONNEES PAR COLLEGES

### afin de pouvoir utiliser la carte sur matplotlib nous avons besoin d'un authentificayion par clé

In [46]:
px.set_mapbox_access_token(mapbox)

#### selectionner les colonnes de données à mettre sur cette première carte

In [47]:
df_data_carte_1 = df_data_carto[['nom_etablissement','ips','taux_de_reussite','latitude','longitude']]
df_data_carte_1.isna().sum()

nom_etablissement      0
ips                  144
taux_de_reussite     144
latitude              90
longitude             90
dtype: int64

### retrait des valeurs NaN et nettoyage des valeurs taux de réussite

In [48]:
# enlever les %%
df_data_carte_1['taux_de_reussite'] = df_data_carte_1['taux_de_reussite'].str.replace("%", " ")
# remplacer la virgule par un point
df_data_carte_1['taux_de_reussite'] = df_data_carte_1['taux_de_reussite'].apply(lambda x : str(x).replace(",", ".")).astype(float)

/var/folders/5n/5_6qtt550_3dxz3f8y6wwjww0000gn/T/ipykernel_38038/1374531501.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_data_carte_1['taux_de_reussite'] = df_data_carte_1['taux_de_reussite'].str.replace("%", " ")
/var/folders/5n/5_6qtt550_3dxz3f8y6wwjww0000gn/T/ipykernel_38038/1374531501.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_data_carte_1['taux_de_reussite'] = df_data_carte_1['taux_de_reussite'].apply(lambda x : str(x).replace(",", ".")).astype(float)


In [112]:
df_data_carte_1.dropna(axis=0, inplace=True)

/var/folders/5n/5_6qtt550_3dxz3f8y6wwjww0000gn/T/ipykernel_12702/2318154728.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_data_carte_1.dropna(axis=0, inplace=True)


In [113]:
df_data_carte_1

,nom_etablissement,ips,taux_de_reussite,latitude,longitude
191,Collège André Pioger,100.3,96.6,48.131339,-0.015090
192,Collège la Noe Lambert,101.7,79.0,47.232017,-1.527280
193,Collège Robert Schuman,93.6,78.1,47.726093,-1.370831
194,Collège Hector Berlioz,110.4,86.8,47.240490,-1.571640
195,Collège Les Sables d'or,123.4,97.2,47.264482,-1.456400
...,...,...,...,...,...
7188,Collège Olympe de Gouges (site de Saint-Hyppol...,103.6,96.9,47.393086,6.772410
7189,Collège Les Villanelles,103.7,83.9,47.483716,6.353580
7190,Collège Pierre Vernotte,102.7,90.7,46.436915,5.721950
7191,Collège René Cassin,109.1,85.6,47.350768,6.352531


In [114]:
df_data_carte_1.isna().sum()

nom_etablissement    0
ips                  0
taux_de_reussite     0
latitude             0
longitude            0
dtype: int64

In [116]:
fig = px.scatter_mapbox(df_data_carte_1, lon='longitude', lat='latitude',color='ips', size='taux_de_reussite',
    color_continuous_scale=px.colors.cyclical.IceFire, size_max=15, zoom=4)
fig.show()

### DataBase ADMIN express pour définir les adresses affiliées aux collèges

In [98]:
r = session.get(f'{api1}datasets/60e678fececb26f4352c4f6a', headers = headers).json()
hrefs_csv = r['resources'][0]['latest']
hrefs_csv
with urlopen(hrefs_csv) as response:
    colleges = pd.read_csv(response, delimiter=';')

In [117]:
colleges = pd.DataFrame(colleges)

In [118]:
# N'ayant malheureusement pas d'information sur les académies autres que
# la Guyane ou la polynésie française, concentrons nous sur la France
# métropolitaine (avec la Corse)
colleges = colleges.query("libelle_academie != 'GUYANE' or libelle_academie != 'POLYNESIE FRANCAISE'")

In [119]:
# Nettoyage
colleges.drop(['code_region', 'libelle_region', 'code_academie','code_departement',
            'code_insee','lieu_dit'], axis=1, inplace=True)

In [120]:
# isoler les secteurs multiples
secteurs_multiples = colleges.query("secteur_unique == 'N'")

liste_colleges = {}

for i in set(secteurs_multiples['code_rne']):
    do =  secteurs_multiples[secteurs_multiples['code_rne'] == i]
    liste_colleges[i]=[ y for y in do['type_et_libelle']]

In [121]:
# Initier la nouvelle colonne
col = []
for i in colleges['code_rne']:
    try:
        col.append(liste_colleges[i])
    except:
        col.append('Secteur unique')


# Ajouter la nouvelle colonne au df
colleges['adresses'] = col

In [122]:
colleges

,libelle_academie,libelle_departement,code_postal,libelle_commune,type_et_libelle,n_de_voie_debut,indice_de_repetition_debut,n_de_voie_fin,indice_de_repetition_fin,parite,code_rne,secteur_unique,adresses
0,AMIENS,SOMME,80300.0,Albert,AVENUE DE LA REPUBLIQUE,1.0,NaN,9999.0,NaN,PI,0801786G,N,"[AVENUE DE LA REPUBLIQUE, RUE ROGER SALENGRO, ..."
1,AMIENS,SOMME,80300.0,Albert,RUE ROGER SALENGRO,1.0,NaN,9999.0,NaN,PI,0801786G,N,"[AVENUE DE LA REPUBLIQUE, RUE ROGER SALENGRO, ..."
2,AMIENS,SOMME,80300.0,Albert,RUE DES ROSES,1.0,NaN,9999.0,NaN,PI,0801375K,N,"[RUE DES ROSES, RUE DE LA STATION, RUE D ULVER..."
3,AMIENS,SOMME,80300.0,Albert,RUE DE LA STATION,1.0,NaN,9999.0,NaN,PI,0801375K,N,"[RUE DES ROSES, RUE DE LA STATION, RUE D ULVER..."
4,AMIENS,SOMME,80300.0,Albert,RUE THIERS,1.0,NaN,9999.0,NaN,PI,0801786G,N,"[AVENUE DE LA REPUBLIQUE, RUE ROGER SALENGRO, ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
353084,NANCY-METZ,MEURTHE-ET-MOSELLE,54410.0,Laneuveville-devant-Nancy,RUE GILBERT BIZE,55.0,NaN,55.0,NaN,I,0542468P,N,"[RUE GUSTAVE FLAUBERT, RUE GUSTAVE FLAUBERT, R..."
353085,NANCY-METZ,MEURTHE-ET-MOSELLE,54410.0,Laneuveville-devant-Nancy,RUE GILBERT BIZE,59.0,NaN,59.0,NaN,I,0542468P,N,"[RUE GUSTAVE FLAUBERT, RUE GUSTAVE FLAUBERT, R..."
353086,NANCY-METZ,MEURTHE-ET-MOSELLE,54410.0,Laneuveville-devant-Nancy,RUE GILBERT BIZE,89.0,NaN,89.0,NaN,I,0542468P,N,"[RUE GUSTAVE FLAUBERT, RUE GUSTAVE FLAUBERT, R..."
353087,NANCY-METZ,MEURTHE-ET-MOSELLE,54410.0,Laneuveville-devant-Nancy,RUE GILBERT BIZE,93.0,NaN,93.0,NaN,I,0542468P,N,"[RUE GUSTAVE FLAUBERT, RUE GUSTAVE FLAUBERT, R..."


### Visualiser en ne gardant que les adresses

In [123]:
# Ne garder que ce que nous souahitons merger
colleges_cleaned = colleges.filter(['code_rne','adresses',"code_postal",'libelle_commune'])

In [ ]:
dispositifs_collèges_df_clean.columns

Index(['code_academie', 'numero_d_etablissement', 'nom_etablissement',
       'libelle_academie', 'type_etablissement', 'statut_public_prive',
       'nombre_d_eleves', 'hebergement', 'ulis', 'segpa',
       'appartenance_education_prioritaire', 'latitude', 'longitude'],
      dtype='object')

In [124]:
# Merge !
all_data = pd.merge(left=df_data_carto, right=colleges_cleaned, left_on='numero_d_etablissement', right_on='code_rne', how='left')

In [ ]:
all_data

,numero_d_etablissement,nom_etablissement,code_academie,libelle_academie,statut_public_prive,nombre_d_eleves,hebergement,ulis,segpa,appartenance_education_prioritaire,ips,taux_de_reussite,latitude,longitude,zone,code_rne,adresses,code_postal,libelle_commune


In [125]:
all_data.drop_duplicates(subset='numero_d_etablissement', inplace=True)

In [126]:
all_data.columns

Index(['numero_d_etablissement', 'nom_etablissement', 'code_academie',
       'libelle_academie', 'statut_public_prive', 'nombre_d_eleves',
       'hebergement', 'ulis', 'segpa', 'appartenance_education_prioritaire',
       'ips', 'taux_de_reussite', 'latitude', 'longitude', 'zone', 'code_rne',
       'adresses', 'code_postal', 'libelle_commune'],
      dtype='object')

In [128]:
all_data_cleaned = all_data.drop(['numero_d_etablissement','code_academie',
    'statut_public_prive','code_rne'], axis=1)

In [129]:
all_data_cleaned.isna().sum()

nom_etablissement                        0
libelle_academie                         0
nombre_d_eleves                          0
hebergement                              0
ulis                                     0
segpa                                    0
appartenance_education_prioritaire       0
ips                                     50
taux_de_reussite                        50
latitude                                 0
longitude                                0
zone                                     0
adresses                              1237
code_postal                           3311
libelle_commune                       1237
dtype: int64

### Export GeoJSON

In [130]:
# columns used for constructing geojson object
features = all_data_cleaned.apply(
    lambda row: Feature(geometry=Point((float(row['longitude']), float(row['latitude'])))),
    axis=1).tolist()

# all the other columns used as properties
properties = all_data_cleaned.drop(['latitude', 'longitude'], axis=1).to_dict('records')

# whole geojson object
# feature_collection = FeatureCollection(features=features, properties=properties[0])

In [ ]:
for el, i in zip(features,properties):
    el['properties']= i
with open('./data/file.geojson', 'w', encoding='utf-8') as f:
    json.dump(features, f, ensure_ascii=False)
